In [3]:
!pip install lxml
!pip install numpy

You are using pip version 19.0.3, however version 20.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from lxml import html
import requests
import numpy as np
# -*- coding: utf-8 -*-

In [5]:
base_url = "https://www.irishtimes.com"
blog_url = "/crosaire-blog"
paginated_url = "?sectionTeaserPage-7.4322291="
crosaire_url = base_url + blog_url + paginated_url

In [6]:
def pull_clues(page_count):
    clues = []
    
    for i in range(page_count):
        page = requests.get(crosaire_url + str(i))
        tree = html.fromstring(page.content)
        links = tree.xpath('''
            //body/div[@id="wrapper"]
            /div[@id="torso"]/div[@class="container"]/div[@class="bt-container"]
            /div[@class="row"]/div[@id="content_left"]/div[@class="row sectionteaser"]
            /div/a''')

        for elt in links:
            clue_page_url = base_url + elt.attrib['href']
            clue_page = requests.get(clue_page_url)
            clue_tree = html.fromstring(clue_page.content)

            clue_list = clue_tree.xpath('''//body/div[@id="wrapper"]
                /div[@id="torso"]/div[@class="container"]/div[@class="bt-container"]
                /div[@class="row"]/div[@id="content_left"]/article[@class="article row"]
                /div[@class="article_holder span8 genre-advice"]/section[@class="article_body"]
                /div[@class="article_bodycopy"]/p''')

            for clue in clue_list:
                clues.append(clue.text_content().replace("â","\'"))
    
    return clues

In [7]:
clues = pull_clues(10)

In [8]:
print(len(clues))

2916


In [9]:
for clue in clues:
    print(clue)
    

8 Teaser from electricians ('teaser' from 'electricians' = lcicin = CLINIC) providing remedies (CLINIC),
9 One paying attention (LISTENER) to line (-L-) from translated (= anagram indicator) entries (entries = -ISTENER) (LISTENER),
10 Those little troublemakers (IMPS) in The Simpsons (in 'the sIMPSons' = IMPS),
11 They're working on site (DEVELOPERS) for those on the make (DEVELOPERS),
12 Shelter for brave (WIGMAM) women (-W-) protected by Wise Guys (Magi = -IG-AM) at work (W-) ' that's a reflection on them all (= reverse all indicator) (WIGWAM),
14 Irish men (Irish 'men' = FIR-) at closing time (closing 'timE' = -E) find way out (DOOR) (FIRE DOOR) is usually closed unless there is an emergency (FIRE DOOR),
15 The New Yorker (AMERICAN) on mission (EMBASSY) (AMERICAN EMBASSY) in foreign office you'd associate with us (US = United States)? (AMERICAN EMBASSY),
18 Fishy (= anagram indicator) suppers (suppers = SUPPRES-) and last drinks (last 'drinkS' = -S) (SUPPRESS) to kee